<a href="https://colab.research.google.com/github/yadavsharn/Diabetic-Retinopathy-Detection/blob/main/Diabetic_retinopathy_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

ds = load_dataset("EslamHasan/APTOS2019DiabeticRetinopathy")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/473 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

data/train-00000-of-00022.parquet:   0%|          | 0.00/435M [00:00<?, ?B/s]

...

data/train-00018-of-00022.parquet:   0%|          | 0.00/262M [00:00<?, ?B/s]

data/train-00019-of-00022.parquet:   0%|          | 0.00/548M [00:00<?, ?B/s]

data/train-00020-of-00022.parquet:   0%|          | 0.00/526M [00:00<?, ?B/s]

data/train-00021-of-00022.parquet:   0%|          | 0.00/515M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3662 [00:00<?, ? examples/s]

In [3]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import efficientnet_b3
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

In [4]:
!pip install opencv-python
import cv2
from PIL import Image
import numpy as np

def enhance_image(pil_img):
    img = np.array(pil_img)
    # Convert to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    # Apply CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    # Merge channels and convert back to RGB
    limg = cv2.merge((cl, a, b))
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    # Optional denoising
    denoised_img = cv2.fastNlMeansDenoisingColored(enhanced_img, None, h=10, hColor=10, templateWindowSize=7, searchWindowSize=21)
    return Image.fromarray(denoised_img)


In [5]:
# Transforms for training and validation
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Custom Dataset class with image enhancement
class APTOSDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None, enhance=False):
        self.dataset = dataset
        self.transform = transform
        self.enhance = enhance

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img = self.dataset[idx]['image']
        label = self.dataset[idx]['label']

        if self.enhance:
            img = enhance_image(img)
        if self.transform:
            img = self.transform(img)

        return img, label


In [8]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)  # weighted sum
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    acc = accuracy_score(all_labels, all_preds)
    return avg_loss, f1, acc


In [ ]:
# Validation function
def eval_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            # ✅ weight loss by batch size
            running_loss += loss.item() * images.size(0)

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # ✅ divide by total dataset size
    avg_loss = running_loss / len(loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    acc = accuracy_score(all_labels, all_preds)
    return avg_loss, f1, acc


# Main training loop example
num_epochs = 25
best_val_f1 = 0.0

for epoch in range(num_epochs):
    train_loss, train_f1, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_f1, val_acc = eval_epoch(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, F1: {train_f1:.4f}, Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, F1: {val_f1:.4f}, Acc: {val_acc:.4f}")

    # Save best model based on validation F1
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "aptos_effnetb3.pth")
        print("model saved")
